In [1]:
# Impor library yang diperlukan
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, expr, when, to_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

In [2]:
# Membuat SparkSession
spark = SparkSession.builder \
    .appName("PredictiveMaintenanceStreaming") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
    .getOrCreate()

print("SparkSession berhasil dibuat.")

:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-81b85ea0-8b69-4027-bbd7-5e29d202b6e3;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downloa

25/07/10 16:17:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


SparkSession berhasil dibuat.


In [9]:
# Mendefinisikan Skema
raw_schema = StructType([
    StructField("timestamp", StringType(), True),
    StructField("machine_id", StringType(), True),
    StructField("vibration", DoubleType(), True),
    StructField("acoustic", DoubleType(), True),
    StructField("temperature", DoubleType(), True),
    StructField("current", DoubleType(), True),
    StructField("IMF_1", DoubleType(), True),
    StructField("IMF_2", DoubleType(), True),
    StructField("IMF_3", DoubleType(), True),
    StructField("label", IntegerType(), True)
])

In [10]:
# Membaca dari topik Kafka
raw_stream_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "raw_sensor_data") \
    .option("startingOffsets", "earliest") \
    .load()

In [11]:
raw_stream_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [12]:
# Parsing dan Transformasi Data
parsed_df = raw_stream_df \
    .select(from_json(col("value").cast("string"), raw_schema).alias("data")) \
    .select("data.*")

In [ ]:
# Transformasi utama
# transformed_df = parsed_df \
#     .withColumn("event_timestamp", to_timestamp(col("datetime"))) \
#     .withColumn("status",
#         when((col("volt") > 250) | (col("volt") < 190), "Voltage Anomaly")
#         .when(col("vibration") > 70, "High Vibration")
#         .otherwise("Normal")
#     )

In [13]:
transformed_df = parsed_df \
    .withColumn("event_timestamp", to_timestamp(col("timestamp"))) \
    .withColumn("status",
        when(col("label") == 1, "Failure Detected")
        .otherwise("Normal")
    )

In [14]:
# Pilih kolom final
final_df = transformed_df.select(
    "event_timestamp", "machine_id", "vibration", "acoustic", 
    "temperature", "current", "status",
    "IMF_1", "IMF_2", "IMF_3",
    "label"
)

In [15]:
# Memastikan transformasi benar sebelum menulis kembali ke Kafka.
# Menulis ke konsol untuk debugging
console_query = final_df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

25/07/10 16:18:55 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-98781038-d8d6-4b88-aa40-4c4751887f63. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/07/10 16:18:55 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [16]:
# Menulis ke topik Kafka yang sudah bersih
kafka_output_df = final_df.select(expr("to_json(struct(*)) AS value"))
kafka_query = kafka_output_df \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("topic", "clean_sensor_data") \
    .option("checkpointLocation", "/tmp/spark_checkpoints/kafka_sensor_writer") \
    .start()

25/07/10 16:18:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------


+-------------------+----------+---------+--------+-----------+-------+----------------+-----+------+-----+-----+
|event_timestamp    |machine_id|vibration|acoustic|temperature|current|status          |IMF_1|IMF_2 |IMF_3|label|
+-------------------+----------+---------+--------+-----------+-------+----------------+-----+------+-----+-----+
|2024-07-01 08:00:00|M01       |0.822    |0.645   |66.85      |13.04  |Normal          |0.196|0.033 |0.0  |0    |
|2024-07-01 08:01:00|M01       |1.398    |0.834   |76.2       |15.08  |Failure Detected|0.345|0.132 |0.001|1    |
|2024-07-01 08:02:00|M01       |0.856    |0.59    |67.03      |12.3   |Normal          |0.187|0.017 |0.002|0    |
|2024-07-01 08:03:00|M01       |0.793    |0.544   |65.04      |11.69  |Normal          |0.196|-0.06 |0.003|0    |
|2024-07-01 08:04:00|M01       |1.279    |0.721   |78.19      |14.84  |Failure Detected|0.33 |-0.115|0.004|1    |
|2024-07-01 08:05:00|M01       |0.782    |0.655   |61.95      |11.56  |Normal          |

-------------------------------------------
Batch: 1
-------------------------------------------
+-------------------+----------+---------+--------+-----------+-------+----------------+-----+------+-----+-----+
|event_timestamp    |machine_id|vibration|acoustic|temperature|current|status          |IMF_1|IMF_2 |IMF_3|label|
+-------------------+----------+---------+--------+-----------+-------+----------------+-----+------+-----+-----+
|2024-07-01 08:29:00|M01       |0.788    |0.611   |64.5       |12.32  |Normal          |0.105|0.093 |0.027|0    |
|2024-07-01 08:30:00|M01       |1.137    |0.862   |74.22      |15.29  |Failure Detected|0.139|-0.148|0.028|1    |
|2024-07-01 08:31:00|M01       |0.75     |0.582   |64.04      |11.75  |Normal          |0.141|0.022 |0.029|0    |
|2024-07-01 08:32:00|M01       |0.937    |0.652   |65.23      |12.25  |Normal          |0.172|-0.038|0.03 |0    |
|2024-07-01 08:33:00|M01       |0.793    |0.57    |59.79      |11.52  |Normal          |0.191|-0.075|0.03

-------------------------------------------
Batch: 2
-------------------------------------------
+-------------------+----------+---------+--------+-----------+-------+------+-----+------+-----+-----+
|event_timestamp    |machine_id|vibration|acoustic|temperature|current|status|IMF_1|IMF_2 |IMF_3|label|
+-------------------+----------+---------+--------+-----------+-------+------+-----+------+-----+-----+
|2024-07-01 08:50:00|M01       |0.735    |0.666   |64.08      |11.89  |Normal|0.134|-0.069|0.042|0    |
|2024-07-01 08:51:00|M01       |0.826    |0.598   |62.46      |11.24  |Normal|0.15 |0.085 |0.043|0    |
+-------------------+----------+---------+--------+-----------+-------+------+-----+------+-----+-----+



...

-------------------------------------------
Batch: 1011
-------------------------------------------
+-------------------+----------+---------+--------+-----------+-------+------+-----+------+------+-----+
|event_timestamp    |machine_id|vibration|acoustic|temperature|current|status|IMF_1|IMF_2 |IMF_3 |label|
+-------------------+----------+---------+--------+-----------+-------+------+-----+------+------+-----+
|2024-07-01 16:15:00|M02       |0.821    |0.582   |67.35      |12.28  |Normal|0.115|-0.107|-0.023|0    |
+-------------------+----------+---------+--------+-----------+-------+------+-----+------+------+-----+



-------------------------------------------
Batch: 1012
-------------------------------------------
+-------------------+----------+---------+--------+-----------+-------+------+-----+-----+------+-----+
|event_timestamp    |machine_id|vibration|acoustic|temperature|current|status|IMF_1|IMF_2|IMF_3 |label|
+-------------------+----------+---------+--------+-----------+-------+------+-----+-----+------+-----+
|2024-07-01 16:16:00|M02       |0.822    |0.621   |60.64      |12.73  |Normal|0.107|0.074|-0.024|0    |
+-------------------+----------+---------+--------+-----------+-------+------+-----+-----+------+-----+

-------------------------------------------
Batch: 1013
-------------------------------------------
+-------------------+----------+---------+--------+-----------+-------+------+-----+------+------+-----+
|event_timestamp    |machine_id|vibration|acoustic|temperature|current|status|IMF_1|IMF_2 |IMF_3 |label|
+-------------------+----------+---------+--------+-----------+------

-------------------------------------------
Batch: 1014
-------------------------------------------
+-------------------+----------+---------+--------+-----------+-------+----------------+-----+-----+------+-----+
|event_timestamp    |machine_id|vibration|acoustic|temperature|current|status          |IMF_1|IMF_2|IMF_3 |label|
+-------------------+----------+---------+--------+-----------+-------+----------------+-----+-----+------+-----+
|2024-07-01 16:18:00|M02       |1.194    |0.839   |78.18      |12.95  |Failure Detected|0.18 |0.111|-0.026|1    |
+-------------------+----------+---------+--------+-----------+-------+----------------+-----+-----+------+-----+

-------------------------------------------
Batch: 1015
-------------------------------------------
+-------------------+----------+---------+--------+-----------+-------+------+-----+------+------+-----+
|event_timestamp    |machine_id|vibration|acoustic|temperature|current|status|IMF_1|IMF_2 |IMF_3 |label|
+------------------

-------------------------------------------
Batch: 1020
-------------------------------------------
+-------------------+----------+---------+--------+-----------+-------+------+-----+-----+-----+-----+
|event_timestamp    |machine_id|vibration|acoustic|temperature|current|status|IMF_1|IMF_2|IMF_3|label|
+-------------------+----------+---------+--------+-----------+-------+------+-----+-----+-----+-----+
|2024-07-01 16:24:00|M02       |0.888    |0.494   |65.26      |11.06  |Normal|0.214|0.029|-0.03|0    |
+-------------------+----------+---------+--------+-----------+-------+------+-----+-----+-----+-----+

-------------------------------------------
Batch: 1021
-------------------------------------------
+-------------------+----------+---------+--------+-----------+-------+------+-----+------+------+-----+
|event_timestamp    |machine_id|vibration|acoustic|temperature|current|status|IMF_1|IMF_2 |IMF_3 |label|
+-------------------+----------+---------+--------+-----------+-------+---

-------------------------------------------
Batch: 1024
-------------------------------------------
+-------------------+----------+---------+--------+-----------+-------+----------------+-----+------+------+-----+
|event_timestamp    |machine_id|vibration|acoustic|temperature|current|status          |IMF_1|IMF_2 |IMF_3 |label|
+-------------------+----------+---------+--------+-----------+-------+----------------+-----+------+------+-----+
|2024-07-01 16:28:00|M02       |1.258    |0.878   |77.05      |14.95  |Failure Detected|0.365|-0.114|-0.034|1    |
+-------------------+----------+---------+--------+-----------+-------+----------------+-----+------+------+-----+

-------------------------------------------
Batch: 1025
-------------------------------------------
+-------------------+----------+---------+--------+-----------+-------+------+-----+------+------+-----+
|event_timestamp    |machine_id|vibration|acoustic|temperature|current|status|IMF_1|IMF_2 |IMF_3 |label|
+-------------

In [ ]:
# Untuk menghentikan semua query:
console_query.stop()
kafka_query.stop()